# FINAL PROJECT

### Mandatory Library

In [2]:
import pandas as pd
import numpy as np
import datetime

## 1. Collecting data

• **What subject is your data about? What is the source of
your data?**
- Data is about Disney movies along with box office success, annual gross income and this dataset aims to seek relationship between box office gross (1) and MPAA ratings (2) in Disney movies
- Source can be found [here](https://data.world/kgarrett/disney-character-success-00-16)
- Description about data has 9 files, 23 columns. But for the sake of this final project we only  use a partial
- What are the trends in the Walt Disney Studio’s box office data? How do certain characters contribute to the success or failure of a movie?
    
• **Do authors of this data allow you to use like this? You
can check the data license**
- The Author has published this data 5 years ago and this data is eligibled for everyone to use in `About this dataset` section
- License: `CC-BY` (3)

• **How did authors collect data?**
- Data is obtained by 4 authors and their references according to their `DisneyReport.pdf`
    - https://www.sugarcane.com/data/walt-disney-animation-studios-films-1
    - http://www.the-numbers.com/movies/distributor/Walt-Disney
    - https://en.wikipedia.org/wiki/List_of_Disney_animated_universe_characters
    - https://en.wikipedia.org/wiki/The_Walt_Disney_Company#Financial_data
- They utilize [import.io](https://www.import.io) to convert raw data into csv file

## 2. Exploring data 
### (often interleaved with preprocessing)

• How many rows and how many columns?

• What is the meaning of each row?

• Are there `duplicated rows`?

• What is the meaning of each column?


• What is the current data type of each column? Are
there columns having `inappropriate data types`?

• With each numerical column, how are values
distributed?
    - What is the percentage of `missing values`?
    - Min? max? Are they `abnormal`?

• With each categorical column, how are values
distributed?
    - What is the percentage of `missing values`?
    - How many different values? Show a few
      Are they `abnormal`?

### Read csv into Dataframe

In [44]:
movies_gross = pd.read_csv('Data/disney_movies_total_gross.csv')
movies_gross.sample(10)

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
190,The Puppet Masters,1994-10-21,Horror,R,8579626,17727017
135,Aladdin,1992-11-11,Comedy,G,217350219,441969178
184,Camp Nowhere,1994-08-26,Comedy,PG,10442641,21576332
423,Sky High,2005-07-29,Adventure,PG,63939454,84088854
291,Holy Man,1998-10-09,Comedy,PG,12069719,21694605
435,The Shaggy Dog,2006-03-10,Comedy,PG,61123569,78667428
482,Up,2009-05-29,Adventure,PG,293004164,329336681
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808
550,The Hundred-Foot Journey,2014-08-08,Romantic Comedy,PG,54235441,55961409
38,Country,1984-09-29,Drama,NaN,8400000,21075000


### How many rows and how many columns?

In [45]:
print("rows:"+ str(movies_gross.shape[0]))
print("columns:"+ str(movies_gross.shape[1]))

rows:579
columns:6


### What is the meaning of column and values?

- `movie_title` : title of a movie  
- `release_date`: the date this movie release  
- `genre`:  Genre of a movie  
- `mpaa_rating`: MPAA ratings of a movie
    - `G`: **General Audiences** All ages admitted. Nothing that would offend parents for viewing by children
    
    - `PG`: **Parental Guidance Suggested** Some material may not be suitable for children. Parents urged to give "parental guidance". May contain some material parents might not like for their young children.
    
    - `PG-13`: **Parents Strongly Cautioned** Some material may be inappropriate for children under 13. Parents are urged to be cautious. Some material may be inappropriate for pre-teenagers.
    
    - `R`: **Restricted** Under 17 requires accompanying parent or adult guardian. Contains some adult material. Parents are urged to learn more about the film before taking their young children with them.
    
    - `Not Rated`
    
- `total_gross`: Total gross of a movie   
- `inflation_adjusted_gross`: inflation of a movies

In [46]:
movies_gross.sample(1)

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
375,25th Hour,2002-12-19,Drama,R,13084595,18325463


### What is the current data type of each column? Are there columns having inappropriate data types?

In [47]:
def open_object_dtype(s):
    dtypes = set()
    dtypes = set(s.apply(type))
    return dtypes

In [48]:
print('movies_title :' + str(open_object_dtype(movies_gross['movie_title'])))
print('release_date :' + str(open_object_dtype(movies_gross['release_date'])))
print('genre :' + str(open_object_dtype(movies_gross['genre'])))
print('mpaa_rating :' + str(open_object_dtype(movies_gross['mpaa_rating'])))
print('total_gross :' + str(open_object_dtype(movies_gross['total_gross'])))
print('inflation_adjusted_gross :' + str(open_object_dtype(movies_gross['inflation_adjusted_gross'])))

movies_title :{<class 'str'>}
release_date :{<class 'str'>}
genre :{<class 'str'>, <class 'float'>}
mpaa_rating :{<class 'str'>, <class 'float'>}
total_gross :{<class 'int'>}
inflation_adjusted_gross :{<class 'int'>}


- As we can see both `Genre` and `mpaa_rating` have the same dtypes of `str` and `float`
- Since we have noticed so far, `release_date` needs converting to Datetime for later use
- Other columns is appropriate 

#### Convert columns into appropriate dtype
- `release_date` converts into datetime dtype
- `genre` converts into string dtype
- `mapaa_rating` converts into string dtype

In [49]:
movies_gross.loc[:,'release_date'] = pd.to_datetime(movies_gross.loc[:,'release_date'],format='%Y/%m/%d')
movies_gross.loc[:,'genre'] = movies_gross.loc[:,'genre'].astype(str)
movies_gross.loc[:,'mpaa_rating'] = movies_gross.loc[:,'mpaa_rating'].astype(str)

### Are there duplicated rows?

In [50]:
have_duplicated_rows = movies_gross.duplicated().any()
have_duplicated_rows

False

### What is the meaning of each row?

Each row is a pack of information of a single movie

In [51]:
movies_gross.sample(1)

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
300,My Favorite Martian,1999-02-12,Comedy,PG,36850101,61150849


### With each numerical column, how are values distributed? - What is the percentage of missing values? - Min? max? Are they abnormal?

- We arrange `release_date`, `total_gross`, `inflation_adjusted_gross` column into numerical group
- Each of columns we're going to merge `missing_ratio`, `min`, and `max` into one dataframe

- `mpaa_rating` column is abnormal because the owner predetermine `nan` values as `string`
-  `total_gross`, `inflation_adjusted_gross` column are abnormal because the owner predetermine `0` as `nan` value

So we have to convert `nan` and `0` into appropriate `np.nan` value

In [52]:
movies_gross.loc[:,'mpaa_rating'].replace('nan',np.nan,inplace=True)
movies_gross.loc[:,'total_gross'].replace(0,np.nan,inplace=True)
movies_gross.loc[:,'inflation_adjusted_gross'].replace(0,np.nan,inplace=True)
movies_gross.loc[:,'genre'].replace('nan',np.nan,inplace=True)

In [53]:
missing = []
missing.append(round(movies_gross.loc[:,'release_date'].isna().sum() / movies_gross.loc[:,'release_date'].size * 100,3))
missing.append(round(movies_gross.loc[:,'total_gross'].isna().sum() / movies_gross.loc[:,'total_gross'].size * 100,3))
missing.append(round(movies_gross.loc[:,'inflation_adjusted_gross'].isna().sum() / movies_gross.loc[:,'inflation_adjusted_gross'].size * 100,3))

minval = []
minval.append(movies_gross.loc[:,'release_date'].min())
minval.append(movies_gross.loc[:,'total_gross'].min())
minval.append(movies_gross.loc[:,'inflation_adjusted_gross'].min())

maxval = []
maxval.append(movies_gross.loc[:,'release_date'].max())
maxval.append(movies_gross.loc[:,'total_gross'].max())
maxval.append(movies_gross.loc[:,'inflation_adjusted_gross'].max())

pd.DataFrame([missing,minval,maxval],index=["missing_ratio", "min", "max"],columns=["release_date", "total_gross", "inflation_adjusted_gross"])


,release_date,total_gross,inflation_adjusted_gross
missing_ratio,0.0,6.910000e-01,6.910000e-01
min,1937-12-21 00:00:00,2.815000e+03,2.984000e+03
max,2016-12-16 00:00:00,9.366622e+08,5.228953e+09


### With each categorical column, how are values distributed? - What is the percentage of missing values? - How many different values? Show a few. Are they abnormal?

- We arrange `movie_title`, `genre`, `mpaa_rating` column into categorical group

In [54]:
missing = []
missing.append(movies_gross.loc[:,'movie_title'].isna().sum() / movies_gross.loc[:,'movie_title'].size * 100)
missing.append(movies_gross.loc[:,'genre'].isna().sum() / movies_gross.loc[:,'genre'].size * 100)
missing.append(movies_gross.loc[:,'mpaa_rating'].isna().sum() / movies_gross.loc[:,'mpaa_rating'].size * 100)

unique = []
unique.append(movies_gross.loc[:,'movie_title'].nunique(dropna=True))
unique.append(movies_gross.loc[:,'genre'].nunique(dropna=True))
unique.append(movies_gross.loc[:,'mpaa_rating'].nunique(dropna=True))

diff_val = []
diff_val.append(movies_gross.loc[:,'movie_title'].dropna().unique())
diff_val.append(movies_gross.loc[:,'genre'].dropna().unique())
diff_val.append(movies_gross.loc[:,'mpaa_rating'].dropna().unique())

pd.DataFrame([missing,unique,diff_val],index=["missing_ratio","Unique",'diff_val'],columns=["movie_title", "genre", "mpaa_rating"])



,movie_title,genre,mpaa_rating
missing_ratio,0.0,2.936097,9.671848
Unique,573,12,5
diff_val,"[Snow White and the Seven Dwarfs, Pinocchio, F...","[Musical, Adventure, Drama, Comedy, Action, Ho...","[G, Not Rated, PG, R, PG-13]"


- We are noticing no abnormal in categroical group

## 3. Asking meaningful questions

Your group needs to give ≥ `the-number-of-group-members`
questions which can be answered with this data. Each
question should be `meaningful` (what are benefits of finding
the answer?) and `not too easy to answer` (e.g., it’s too easy if
we just need one line of code to get the answer). Your
group should focus more on `the quality of questions` than
the quantity.

In notebook file, with each question, your group needs to
present:

• What is the question?
- Which genres give out most total gross and least in total (thể loại nào đem lại lợi nhuận cao nhất và thấp nhất)
- Kiểm tra các nhân vật nào được yêu thích nhất và phim thuộc vào thể loại nào và đạo diễn với voice actor là ai (nếu có) **(CHƯA LÀM)**
- Xem xét hiệu quả kinh doanh của các thể loại phim hành động, phiêu lưu trước thế kỷ 20 và sau thế kỷ 20 (cột inflation)
- Xu hướng thể loại phim yêu thích của khán giả có thay đổi qua thời gian hay không ?


• What are benefits of finding the answer?
- To examine which genres should Disney zero in on to produce the best quality, story for upcoming films  
(xem xét thể loại nào mà Disney tập trung vào để cho ra những bộ phim kế tiếp có chất lượng, câu truyện tốt nhất, và tránh những thể loại mang lại thu nhập thấp)
- Để xem xét các quality của phim tập trung vào voice actor và đạo diễn nào và có thể phim kế tiếp trùng genres thì ta có thể mời người đó làm đạo diễn 
- Để xem có khác biệt về thu thập cho chúng ta biết theo thời gian disney có phải ngày càng làm phim tệ hay phim làm ra hoặc hợp tác ngày càng hay và vẫn thu hút được mọi người

In [55]:
movies_gross.head()

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485.0,5.228953e+09
1,Pinocchio,1940-02-09,Adventure,G,84300000.0,2.188229e+09
2,Fantasia,1940-11-13,Musical,G,83320000.0,2.187091e+09
3,Song of the South,1946-11-12,Adventure,G,65000000.0,1.078511e+09
4,Cinderella,1950-02-15,Drama,G,85000000.0,9.206087e+08


In [56]:
df = pd.read_csv('Data/disney_voice_actors.csv')
df.head(5)

,character,voice_actor,movie
0,Abby Mallard,Joan Cusack,Chicken Little
1,Abigail Gabble,Monica Evans,The Aristocats
2,Abis Mal,Jason Alexander,The Return of Jafar
3,Abu,Frank Welker,Aladdin
4,Achilles,None,The Hunchback of Notre Dame


In [57]:
df.shape

(935, 3)

In [58]:
b, counts = np.unique(df.loc[:,'voice_actor'],return_counts=True)
sort_idx = np.argsort(counts,kind='stable')



## 4. Preprocessing + analyzing data to answer each question

With each question:

• Does it need to have preprocessing step, and if yes,
how does your group preprocess?

• Text: sketch steps `clearly` so that readers can
understand how your group preprocesses even without
reading code

• Code: implement sketched steps. Your group should
also try to write code `clearly` (choose good variable
names, comment where should be commented, don’t
let a line too long)

• How does your group analyze data to answer the
question?

### **Câu hỏi 1:**
**Which genres give out most total gross and least in total (thể loại nào đem lại lợi nhuận cao nhất và thấp nhất)**

**Các bước thực hiện:**
- Gom nhóm những bộ film cùng thể loại
- Tìm ra doanh thu lớn nhất và thể loại đó
- Tim ra doanh thu nhỏ nhất và thể loại đó

In [59]:
# Cau 1
genre_total_gross = movies_gross.groupby('genre')['total_gross'].sum()
genre_max = genre_total_gross[genre_total_gross == genre_total_gross.max()]
genre_min = genre_total_gross[genre_total_gross == genre_total_gross.min()]

print('Thể loại film đem lại lợi nhuận cao nhất là:',genre_max.index[0])
print('Với tổng doanh thu:',genre_max.values[0],'USD')
print()
print('Thể loại film đem lại lợi nhuận thấp nhất là:',genre_min.index[0])
print('Với tổng doanh thu:',genre_min.values[0],'USD')

Thể loại film đem lại lợi nhuận cao nhất là: Adventure
Với tổng doanh thu: 16389069453.0 USD

Thể loại film đem lại lợi nhuận thấp nhất là: Horror
Với tổng doanh thu: 87068872.0 USD


In [60]:
movies_gross.head(10)

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485.0,5.228953e+09
1,Pinocchio,1940-02-09,Adventure,G,84300000.0,2.188229e+09
2,Fantasia,1940-11-13,Musical,G,83320000.0,2.187091e+09
3,Song of the South,1946-11-12,Adventure,G,65000000.0,1.078511e+09
4,Cinderella,1950-02-15,Drama,G,85000000.0,9.206087e+08
5,"20,000 Leagues Under the Sea",1954-12-23,Adventure,NaN,28200000.0,5.282800e+08
6,Lady and the Tramp,1955-06-22,Drama,G,93600000.0,1.236036e+09
7,Sleeping Beauty,1959-01-29,Drama,NaN,9464608.0,2.150583e+07
8,101 Dalmatians,1961-01-25,Comedy,G,153000000.0,1.362871e+09
9,The Absent Minded Professor,1961-03-16,Comedy,NaN,25381407.0,3.100946e+08


### Câu hỏi 3:
**Xem xét hiệu quả kinh doanh của các thể loại phim hành động, phiêu lưu trước thế kỷ 20 và sau thế kỷ 20**

**Các bước thực hiện:**
- Lọc ra những phim có thể loại hành động và phiêu lưu
- Tạo thêm cột `year` được trích xuất từ cột `release_date`
- Gom nhóm các phim trước năm 2000 và sau năm 2000
- Tính tổng doanh thu dựa trên cột `inflation_adjusted_gross`
- Tính doanh thu trung bình của mỗi phim trong 2 giai đoạn
- Rút ra nhận xét

In [61]:
movies_gross['year'] = pd.DatetimeIndex(movies_gross['release_date']).year

options = ['Action','Adventure']
df = movies_gross[movies_gross['genre'].isin(options)]

before_20th_df = df[df['year'] <= 2000] 
after_20th_df = df[df['year'] > 2000]
total_gross_before20th = before_20th_df['inflation_adjusted_gross'].sum()
total_gross_after20th = after_20th_df['inflation_adjusted_gross'].sum()

num_films_before20th = before_20th_df['movie_title'].count()
num_films_after20th = after_20th_df['movie_title'].count()

print('Xét trên 2 thể loại hành động và phiêu lưu ta có:','\n')
print('Tổng doanh thu trước thế kỷ 20:',total_gross_before20th)
print('Số phim trước thế kỷ 20:',num_films_before20th)
print('Thời gian ra mắt:',before_20th_df['year'].min(),'->',before_20th_df['year'].max(),'\n')

print('Tổng doanh thu sau thế kỷ 20:',total_gross_after20th)
print('Số phim sau thế kỷ 20:',num_films_after20th)
print('Thời gian ra mắt:',after_20th_df['year'].min(),'->',after_20th_df['year'].max(),'\n')

print('Doanh thu trung bình của mỗi phim trước thế kỷ 20:',total_gross_before20th/num_films_before20th)
print('Doanh thu trung bình của mỗi phim sau thế kỷ 20:',total_gross_after20th/num_films_after20th)
print('(Thể loại phim hành động, phiêu lưu có sự tăng trưởng sau thế kỷ 20)','\n')

ratio = (total_gross_after20th/num_films_after20th) / (total_gross_before20th/num_films_before20th) - 1
print('Tỉ lệ tăng trưởng doanh thu:',ratio.round(4)*100,'%')



Xét trên 2 thể loại hành động và phiêu lưu ta có: 

Tổng doanh thu trước thế kỷ 20: 12152462471.0
Số phim trước thế kỷ 20: 75
Thời gian ra mắt: 1940 -> 2000 

Tổng doanh thu sau thế kỷ 20: 17907740473.0
Số phim sau thế kỷ 20: 94
Thời gian ra mắt: 2001 -> 2016 

Doanh thu trung bình của mỗi phim trước thế kỷ 20: 162032832.94666666
Doanh thu trung bình của mỗi phim sau thế kỷ 20: 190507877.3723404
(Thể loại phim hành động, phiêu lưu có sự tăng trưởng sau thế kỷ 20) 

Tỉ lệ tăng trưởng doanh thu: 17.57 %


**Nhận xét:**
- Hiệu quả kinh doanh của hai thể loại phim phiêu lưu, hành động có sự tăng trưởng mạnh
- Tăng trưởng về số lượng:
    - Trước thế kỷ 20, trong **60 năm** ra **75 phim** 
    - Sau thế kỷ 20, trong vòng **15 năm** ra tận **94 phim** 
- Tăng trưởng về doanh thu:
    - Tỉ lệ tăng trưởng doanh thu trên mỗi bộ phim **tăng 17,57 %**

### Câu hỏi 4:
**Xu hướng thể loại phim yêu thích của khán giả có thay đổi qua thời gian hay không ?**
- Để tra lời câu hỏi này, thì em sẽ chia ra từng giai đoạn để kiểm tra( 1 giai đoạn = 5 năm). Với mỗi chu kỳ, ta sẽ kiểm tra xem thể loại phim nào được phát hành nhiều nhất. Qua đó xem xét xu hướng của khán giả.


- **Các bước thực hiện:**
    - Thêm cột `phase` được chia trên cột `year`
    - Những năm `< 1940` được nhập vào giai đoạn đầu tiên
    - Những năm `> 2015` được nhập vào giai đoạn cuối cùng ( Vì dữ liệu chỉ đến năm 2016)
    - Tìm thể loại phim được ra mắt nhiều nhất trong mỗi giai đoạn
    - Xem xét xu hướng thể loại phim được yêu thích

In [149]:
df = movies_gross
df['phase'] = ((df['year'] - 1940) / 5).apply(int)
df.loc[df['year'] <1940, 'phase'] = 0
df.loc[df['year'] >2015, 'phase'] = 15

df = df.groupby(['phase','genre'])['genre'].count()
df = df.unstack(1)

df = df.idxmax(axis=1).to_frame()
df.columns = ['genre_trend']
df = df.reset_index()
df['phase'] = (df['phase']*5+1940).apply(int).apply(str) + ' - ' + (df['phase']*5+1944).apply(int).apply(str)

df['phase'][0] = '1937 - 1944'
df['phase'][15] = '2015 - 2016'
df

,phase,genre_trend
0,1937 - 1944,Musical
1,1945 - 1949,Adventure
2,1950 - 1954,Adventure
3,1955 - 1959,Drama
4,1960 - 1964,Comedy
5,1965 - 1969,Comedy
6,1970 - 1974,Musical
7,1975 - 1979,Comedy
8,1980 - 1984,Drama
9,1985 - 1989,Comedy


**Nhận xét:**
- Thể loại phim yêu thích có xu hướng dịch chuyển qua từng giai đoạn
- Giai đoạn 1937 - 1959, có thể giai đoạn này chưa định hình được xu hướng vì có 4 giai đoạn nhưng có đến 3 thể loại phim
- Giai đoạn 1960 - 2004, là giai đoạn thành công của thể loại phim hài khi chiếm phần đa số
- Giai đoạn 2005 - 2016, thể loại phim được khán giả quan tâm yêu thích nhất là thể loại phim phiêu lưu, có thể nói thể loại phim này đang thống trị

## 5. Reflection

• Each member: What difficulties have you encountered?

• Each member: What have you learned?

• Your group: If you had more time, what would you do?

## 6. References and Footnote
To finish this project, what materials have you consulted?



(1): `Box office gross` -> Revenue generated from ticket sales (receipts) including any taxes and other levies.  
(2): `MPAA ratings` -> The MPAA rating system is a voluntary film-rating system created by the Motion Picture Association of America (MPAA) and the National Association of Theatre Owners (NATO) to determine a movie's suitability for audiences and age groups based on content.  
(3): `CC-BY` -> it is the most open license. It allows the user to redistribute, to create derivatives, such as a translation, and even use the publication for commercial activities, provided that appropriate credit is given to the author (BY) and that the user indicates whether the publication has been changed.